### Django 프로젝트 기본 셋업 : 최소한의 settings 설정
직렬화 연습을 위한 장고 프로젝트 세팅해서 모델 돌려보기

In [1]:
import django
import os

SECRET_KEY = 'tada'  # 임의 문자열 가능
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': ':memory:',
    }
}

ROOT_URLCONF = '__main__'
urls = []
os.environ['DJANGO_SETTINGS_MODULE'] = '__main__'

django.setup()

### Form 클래스를 정의하고 사용하기

In [2]:
from django import forms

def odd_validator(value):
    if value % 2 == 0:
        raise forms.ValidationError("내가 짝수라니!!")

class QuizForm(forms.Form):
    answer = forms.IntegerField(validators=[odd_validator])

In [3]:
data = {'answer': 10}

In [4]:
form = QuizForm(data)

In [6]:
form.is_valid()  # 짝수이기 때문에 유효성 검사 실패

False

In [7]:
form.errors

{'answer': ['내가 짝수라니!!']}

### Model 클래스 정의하고 사용하기

In [9]:
from django.db import models

class Post(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        app_label = 'api'  # 앱이 따로 없으므로 app label을 필히 지정
        
    def __str__(self):
        return self.title

#### migration 여건이 안되기 때문에 모델 클래스 내역 그대로 raw SQL로 DB 테이블 생성

In [10]:
from django.db import connection

table_name = Post._meta.db_table

In [12]:
with connection.cursor() as cursor:
    cursor.execute('''
    CREATE TABLE "{}"
        ('id' integer NOT NULL PRIMARY KEY AUTOINCREMENT,
        'title' varchar(100) NOT NULL,
        "content" text NOT NULL,
        "created_at" datetime NOT NULL,
        "updated_at" datetime NOT NULL);
    '''.format(table_name))

In [13]:
# table 확인 : 앱이름_모델명, 
with connection.cursor() as cursor:
    cursor.execute("select name from sqlite_master where type='table';")
    for row in cursor.fetchall():
        print(row)

('api_post',)
('sqlite_sequence',)


### Post 모델을 통한 DB에 데이터 추가

In [14]:
Post.objects.create(
    title='The Year in Artful Music: Sounds for Contemplation',
    content='Music is art in all cases, and any energy devoted to segregation is misspent. But then, at the same time, there are manifestations of music that make the unity more clear—easier to acknowledge, accept, and affirm. Yet even then, subjectivity enters in, such that the notion of distinctions starts to reveal itself as farce. And then we arrive once again at the essential truth that we started with: music is art—and on and on and on.'
)

<Post: The Year in Artful Music: Sounds for Contemplation>

In [15]:
Post.objects.create(
    title='Simone Forti, Al Di Là (Saltern)',
    content='Drawn from a show titled “Sounding” at the Box in Los Angeles, this compilation by the dancer/choreographer/etc. Simone Forti features aural artworks dating from the 1960s to the ’80s. One is a sung song accompanied by a shaking can of nails, as performed in Yoko Ono’s loft in 1961; another is the result of “a slide whistle equipped with a stylus.” The four-part track “Bottom” includes vocals by La Monte Young and Marian Zazeela, and then there’s “Dal Di Là,” about which Forti writes in the liner notes: “I made this song towards the end of a period of performing stoned on weed. It roughly translates: I’m awaiting a song from afar, from afar, a song of goodbye from afar. For now I’ve seen the game I was playing, slowly leaving the earth and drifting far among the stars.”'
)

<Post: Simone Forti, Al Di Là (Saltern)>

In [16]:
Post.objects.create(
    title='Technicolor Paradise: Rhum Rhapsodies & Other Exotic Delights',
    content='There’s not much of an art angle to this one, but Technicolor Paradise is probably what I listened to more than anything else this year. '
)

<Post: Technicolor Paradise: Rhum Rhapsodies & Other Exotic Delights>

In [17]:
Post.objects.all()

<QuerySet [<Post: The Year in Artful Music: Sounds for Contemplation>, <Post: Simone Forti, Al Di Là (Saltern)>, <Post: Technicolor Paradise: Rhum Rhapsodies & Other Exotic Delights>]>

In [23]:
for post in Post.objects.all():
    print(post.id, post.title, ':', len(post.content), '글자')

1 The Year in Artful Music: Sounds for Contemplation : 433 글자
2 Simone Forti, Al Di Là (Saltern) : 780 글자
3 Technicolor Paradise: Rhum Rhapsodies & Other Exotic Delights : 137 글자


### ModelForm 설정

In [18]:
class PostForm(forms.ModelForm):
    class Meta:
        model = Post
        fields = '__all__'

In [19]:
form = PostForm({'title': 'hello'})

In [20]:
form.is_valid()

False

In [21]:
form.errors

{'content': ['This field is required.']}

### Django JSON 직렬화
장고 JSON 직렬화는 파이썬 표준 라이브러리 json 모듈을 좀더 발전시킨 형태로 제공 : DjangoJSONEncoder

DjangoJSONENcoder는 파이썬 json.JSONENcoder를 상속받아 time 타입의 직렬화를 추가로 지원
+ datetime.datetime
+ datetime.date
+ datetime.time
+ datetime.timedelta
+ datetime.Decimal, uuid.UUID

그러나 장고 테이터타입인 QuerySet과 Model 인스턴스에 대한 직렬화는 지원하지 않음. 그리고 실제 웹 애플리케이션 개발에서는 JSON 직렬화 일이 별로 없음. 이 부분을 djangorestframework에서 지원해 줌

#### DjangoJSONENcoder 실행

In [25]:
import json
from django.core.serializers.json import DjangoJSONEncoder

In [26]:
data = Post.objects.all()
data

<QuerySet [<Post: The Year in Artful Music: Sounds for Contemplation>, <Post: Simone Forti, Al Di Là (Saltern)>, <Post: Technicolor Paradise: Rhum Rhapsodies & Other Exotic Delights>]>

In [27]:
# 따라서.... 장고 QuerySet을 넘기면 예외 발생
json.dumps(data, cls=DjangoJSONEncoder)

TypeError: Object of type 'QuerySet' is not JSON serializable

###### QuerySet을 파이썬 표준데이터 타입의 값으로 한땀 한땀 직접 변환도 할 수는 있음  ==> json 모듈이 하는 일을 직접 하는 것임

In [31]:
data = [
    {'id': post.id, 'title': post.title, 'content': post.content}
    for post in Post.objects.all()
]
json.dumps(data, cls=DjangoJSONEncoder, ensure_ascii=False)  
# ensure_ascii 를 생략하면 코드값만 출력됨
# cls=DjangoJSONEncoder 없어도 작동

'[{"id": 1, "title": "The Year in Artful Music: Sounds for Contemplation", "content": "Music is art in all cases, and any energy devoted to segregation is misspent. But then, at the same time, there are manifestations of music that make the unity more clear—easier to acknowledge, accept, and affirm. Yet even then, subjectivity enters in, such that the notion of distinctions starts to reveal itself as farce. And then we arrive once again at the essential truth that we started with: music is art—and on and on and on."}, {"id": 2, "title": "Simone Forti, Al Di Là (Saltern)", "content": "Drawn from a show titled “Sounding” at the Box in Los Angeles, this compilation by the dancer/choreographer/etc. Simone Forti features aural artworks dating from the 1960s to the ’80s. One is a sung song accompanied by a shaking can of nails, as performed in Yoko Ono’s loft in 1961; another is the result of “a slide whistle equipped with a stylus.” The four-part track “Bottom” includes vocals by La Monte Y

In [32]:
mydata = ['안녕', '파이썬']
json.dumps(mydata)

'["\\uc548\\ub155", "\\ud30c\\uc774\\uc36c"]'

In [33]:
json.dumps(mydata, ensure_ascii=False)

'["안녕", "파이썬"]'

##### 위의 두가지 형태 모두 지원 가능하나 가독성을 위해 ensure_ascii 지정

## 커스텀 JSON ENcoder 정의
DjangoJSONENcoder 확장하여 Query타입(tuple타입으로 변환), Post타입(dict타입으로 변환)

In [36]:
from django.core.serializers.json import DjangoJSONEncoder
from django.db.models.query import QuerySet


class MYJSONENcoder(DjangoJSONEncoder):
    def default(self, obj):
        if isinstance(obj, QuerySet):
            return tuple(obj)
        elif isinstance(obj, Post):
            return {'id': obj.id, 'title': obj.title, 'content': obj.content}
        return super().default(obj)
    
data = Post.objects.all()

json.dumps(data, cls=MYJSONENcoder, ensure_ascii=False)

'[{"id": 1, "title": "The Year in Artful Music: Sounds for Contemplation", "content": "Music is art in all cases, and any energy devoted to segregation is misspent. But then, at the same time, there are manifestations of music that make the unity more clear—easier to acknowledge, accept, and affirm. Yet even then, subjectivity enters in, such that the notion of distinctions starts to reveal itself as farce. And then we arrive once again at the essential truth that we started with: music is art—and on and on and on."}, {"id": 2, "title": "Simone Forti, Al Di Là (Saltern)", "content": "Drawn from a show titled “Sounding” at the Box in Los Angeles, this compilation by the dancer/choreographer/etc. Simone Forti features aural artworks dating from the 1960s to the ’80s. One is a sung song accompanied by a shaking can of nails, as performed in Yoko Ono’s loft in 1961; another is the result of “a slide whistle equipped with a stylus.” The four-part track “Bottom” includes vocals by La Monte Y

###### django-rest-framework에서도 위와 마찬가지로 커스텀 JSON Encoder를 만드는 방식으로 JSON 인코딩을 처리함